In [44]:
import collections
import os
import sys
import codecs
import numpy as np
import tensorflow as tf

start_token = 'G'
end_token = 'E'

data_dir = '~/Documents/fastai-notes/deeplearning1/rnn/text/'
%cd $data_dir
filename = 'gucheng_compiled.txt'

poems = []
with open(filename, 'r', encoding='utf-8') as f:
    for line in f.readlines():
        try:
            title, content = line.strip().split(':')
            content = content.replace(' ', '')
            if len(content) <= 10:
                continue
            content = start_token + content + end_token
            poems.append(content)
        except ValueError as e:
            pass
poems = sorted(poems, key=lambda line: len(line))

/Users/PeiYingchi/Documents/fastai-notes/deeplearning1/rnn/text


In [45]:
poems[:10]

['G连睡梦的路，都难以到达E',
 'G剪下一页页生活的片断；E',
 'G像一枚，毫不掩饰的果实E',
 'G他似笑非笑，说，你早。E',
 'G没有回答，表情似笑非笑E',
 'G飞来又飞去，飞高又飞低E',
 'G因为有问题，因为有秘密E',
 'G一阵阵快乐在尾巴上跳荡E',
 'G再没有海岸，再没有灯火E',
 'G干燥的红星星，全都脱落E']

In [46]:
all_words = []
for poem in poems:
    all_words += [word for word in poem]
# 这里根据包含了每个字对应的频率
counter = collections.Counter(all_words)
count_pairs = sorted(counter.items(), key=lambda x: -x[1])
words, _ = zip(*count_pairs)


In [47]:
# 取前多少个常用字
words = words[:len(words)] + (' ',)
# 每个字映射为一个数字ID
word_int_map = dict(zip(words, range(len(words))))
word2idfunc = lambda word:  word_int_map.get(word,len(words))
poems_vector = [list(map(word2idfunc, poem)) for poem in poems]

In [48]:
words[:5]

('，', '的', '在', '一', 'G')

In [49]:
#batch-wise padding:do padding to the same size(sequence length) of each batch
batch_size = 16
n_batch = (len(poems_vector)-1) // batch_size
X_data,Y_data = [],[]
for i in range(n_batch):
    cur_vecs = poems_vector[i*batch_size:(i+1)*batch_size]
    current_batch_max_length = max(map(len,cur_vecs))
    batch_matrix = np.full((batch_size,current_batch_max_length),word2idfunc(" "),np.int32)
    for j in range(batch_size):
        batch_matrix[j,:len(cur_vecs[j])] = cur_vecs[j]
    x = batch_matrix
    X_data.append(x)
    y = np.copy(x)
    y[:,:-1] = x[:,1:]
    # print x
    # print y
    Y_data.append(y)

In [50]:
#build rnn
vocab_size = len(words)+1
#input_size:(batch_size,feature_length)
input_sequences = tf.placeholder(tf.int32,shape=[batch_size,None])
output_sequences = tf.placeholder(tf.int32,shape=[batch_size,None])

In [57]:
def build_rnn(hidden_units=128, layers=2):
    # embedding
    with tf.variable_scope("embedding", reuse=True):
        embedding = tf.get_variable("embedding", [vocab_size, hidden_units], dtype=tf.float32)
        input = tf.nn.embedding_lookup(embedding, input_sequences)
        
    basic_cell = tf.nn.rnn_cell.BasicLSTMCell(hidden_units, state_is_tuple=True)
    stack_cell = tf.nn.rnn_cell.MultiRNNCell([basic_cell]*layers)
    _initial_state = stack_cell.zero_state(batch_size, tf.float32)
    outputs, state = tf.nn.dynamic_rnn(stack_cell, input, initial_state=_initial_state, dtype=tf.float32)
    outputs = tf.reshape(outputs, [-1, hidden_units])
    
    with tf.variable_scope('softmax'):
        softmax_w = tf.get_variable('softmax_w', [hidden_units, vocab_size])
        softmax_b = tf.get_variable('softmax_b', [vocab_size])
        logits = tf.matmul(outputs, softmax_w)+softmax_b
        
    probs = tf.nn.softmax(logits)
    return logits, probs, stack_cell, _initial_state, state

In [58]:
def train(reload=True):
    logits, probs,_,_,_ = build_rnn()

    targets = tf.reshape(output_sequences,[-1])

    loss = tf.nn.seq2seq.sequence_loss_by_example([logits], [targets], 
        [tf.ones_like(targets, dtype=tf.float32)],len(words))
    cost = tf.reduce_mean(loss)

    learning_rate = tf.Variable(0.002, trainable=False)
    tvars = tf.trainable_variables()
    grads, _ = tf.clip_by_global_norm(tf.gradients(cost, tvars), 5)
    optimizer = tf.train.AdamOptimizer(learning_rate)
    train_op = optimizer.apply_gradients(zip(grads, tvars))

    global_step = 0
    with tf.Session() as sess:
        sess.run(tf.initialize_all_variables())
        saver = tf.train.Saver(write_version=tf.train.SaverDef.V2)

        if reload:
            module_file = tf.train.latest_checkpoint('.')
            sess = saver.restore(module_file)
            print("reload sess")

        for epoch in range(50):
            print("learning_rate decrease")
            if global_step%80==0:
                sess.run(tf.assign(learning_rate, 0.002 * (0.97 ** epoch)))
            epoch_steps =  len(zip(X_data,Y_data))
            for step,(x,y) in enumerate(zip(X_data,Y_data)):
                global_step = epoch * epoch_steps + step
                _, los  = sess.run([train_op, cost], feed_dict={
                    input_sequences:x,
                    output_sequences:y,
                    })
                print("epoch:%d steps:%d/%d loss:%3f" % (epoch,step,epoch_steps,los))
                if global_step%100==0:
                    print("save model")
                    saver.save(sess,"peotry",global_step=epoch)


def write_poem():
    def to_word(weights):
        t = np.cumsum(weights)
        s = np.sum(weights)
        sample = int(np.searchsorted(t, np.random.rand(1)*s))
        print("sample:",sample)
        print("len Words:",len(words))
        return words[sample]

    logits, probs,stack_cell, _initial_state, last_state = build_rnn()
    with tf.Session() as sess:
        sess.run(tf.initialize_all_variables())
        saver = tf.train.Saver(write_version=tf.train.SaverDef.V2)
        module_file = tf.train.latest_checkpoint('.')
        print("load:",module_file)
        saver.restore(sess,module_file)

        _state = sess.run(stack_cell.zero_state(1,dtype=tf.float32))

        x = np.array([[word2idfunc('[')]])

        prob_, _state = sess.run([probs,last_state],feed_dict={input_sequences:x,_initial_state:_state})

        word = to_word(prob_)

        poem = ''

        import time
        while word != ']':
            poem += word
            x = np.array([[word2idfunc(word)]])
            [probs_, _state] = sess.run([probs, last_state], feed_dict={input_sequences: x, _initial_state: _state})
            word = to_word(probs_)
            # time.sleep(1)

    return poem



In [59]:
print(write_poem())

Instructions for updating:
Use `tf.global_variables_initializer` instead.
load: None
INFO:tensorflow:Restoring parameters from None


SystemError: <built-in function TF_Run> returned a result with an error set